In [3]:
import sys
import numpy as np
import pandas as pd
import os
from IPython import display
import time
import re
import string
%matplotlib inline
path = '/vol/fob-vol7/nebenf21/rohjehyu/Downloads/' 

-General Processing and some dataset specific processing functions.
-Preprocessing is not extensively done for this fake news classifcation task. 
-The original datasets download link is included in the Appendix. 
-The processed datasets are included in the submitted USB. 

In [4]:
def clean_text(data):  #general cleaning.very minimized to not lose any context. 
    clean=[]
    for i in data:
        #to lowercase
        i = i.lower()
        
        #'s
        i = re.sub('\'s'," ", i) 
        i = re.sub('\’s'," ", i) 
        i = re.sub('n\’t'," not", i)               

        # get rid of urls
        i = re.sub('https?://\S+|www\.\S+', ' ', i)

        # Replace the over spaces
        i = re.sub('\s{2,}', " ", i)
        
        clean.append(i)
    return clean

In [5]:
def clean_isot(data):  #for the ISOT dataset in general
    clean=[]
    for i in data:
        #to lowercase
        i = i.lower()
        
        #ISOT real
        i = re.sub('u\.s\.', 'united states', i) # u.s.
        
        #'s
        i = re.sub('\'s'," ", i) 
        i = re.sub('\’s'," ", i) 
        i = re.sub('n\’t'," not", i)               

        # get rid of urls
        i = re.sub('https?://\S+|www\.\S+', ' ', i)

        # Replace the over spaces
        i = re.sub('\s{2,}', " ", i)
        
        clean.append(i)
    return clean

In [6]:
def clean_cc(data):   #handling some unicode errors, etc for combined corpus dataset 
    clean=[]
    for i in data:
        #CC - unicode error
        i = re.sub('â€™', " ", i)
        i = re.sub('Â\xa0'," ", i)
        i = re.sub('â€\x9d'," ", i)
        i = re.sub('â€œ'," ", i) 
        i = re.sub('â€“'," ", i) 
        
        #to lowercase
        i = i.lower()
        
        #'s
        i = re.sub('\'s'," ", i) 
        i = re.sub('\’s'," ", i) 
        i = re.sub('n\’t'," not", i)               

        # get rid of urls
        i = re.sub('https?://\S+|www\.\S+', ' ', i)

        # Replace the over spaces
        i = re.sub('\s{2,}', " ", i)
        
        clean.append(i)
    return clean

In [7]:
def clean_covid(data):  #as corona, covid is frequently in the text, if the words "corona" "virus" were combined, 
                        #I added a space between them to give uniformity 
    clean=[]
    for i in data:
        #to lowercase
        i = i.lower()
        
        #COVID TWEET
        i = re.sub('covid', "covid ", i) 
        i = re.sub('corona', "corona ", i)
        i = re.sub('virus', "virus ", i)
        
        #'s
        i = re.sub('\'s'," ", i) 
        i = re.sub('\’s'," ", i) 
        i = re.sub('n\’t'," not", i)               

        # get rid of urls
        i = re.sub('https?://\S+|www\.\S+', ' ', i)

        # Replace the over spaces
        i = re.sub('\s{2,}', " ", i)
        
        clean.append(i)
    return clean

In [8]:
def isot_fake(data):  # for ISOT fake news, removing the last sentence image link. 
    clean=[]
    for i in data:
        if i.rsplit('.')[-1]=='':
            n=-2
        else:
            n=-1
        i='.'.join(i.rsplit('.')[:n])
        #remove image link
        i = re.sub("pic.twitter.com/*\S+", " ", i)        
        clean.append(i)
    return clean

def isot_real(data): #for ISOT real news, remove the publisher data in the beginning
    clean=[]
    for i in data:
        #to lowercase
        i = i.lower()
        i = re.sub('u\.s\.', 'united states', i) # u.s.
        #remove publisher in the beginning
        i = re.sub('^.*?-', '', i)        
        clean.append(i)
    return clean

ISOT

In [17]:
train_data = pd.read_csv(path + 'Fake1.csv', encoding="UTF-8",low_memory=False)
train_data['labels']=1
train_data['text']=isot_fake(train_data['text'])#cleaning

train_data2 = pd.read_csv(path + 'True1.csv', encoding="UTF-8",low_memory=False)
train_data2['labels']=0
train_data2['text']=isot_real(train_data2['text'])#cleaning

train_data=pd.concat([train_data, train_data2], ignore_index=True)


train_data = train_data.loc[:, ~train_data.columns.str.contains('^Unnamed')]
train_data=train_data.convert_dtypes()
train_data[["date"]] = train_data[["date"]] .apply(lambda x:pd.to_datetime(x,errors='coerce',format="%Y-%m-%d"))


train_data['text']=train_data['title']+train_data['text'] #concatenating title and text column
train_data['text']=clean_isot(train_data['text'])#cleaning


train_data.to_csv(path + 'ISOT_may.csv')

In [18]:
train_data

,title,text,subject,date,labels
0,Donald Trump Sends Out Embarrassing New Year’...,donald trump sends out embarrassing new year ...,News,2017-12-31,1
1,Drunk Bragging Trump Staffer Started Russian ...,drunk bragging trump staffer started russian ...,News,2017-12-31,1
2,Sheriff David Clarke Becomes An Internet Joke...,sheriff david clarke becomes an internet joke...,News,2017-12-30,1
3,Trump Is So Obsessed He Even Has Obama’s Name...,trump is so obsessed he even has obama name c...,News,2017-12-29,1
4,Pope Francis Just Called Out Donald Trump Dur...,pope francis just called out donald trump dur...,News,2017-12-25,1
...,...,...,...,...,...
44914,'Fully committed' NATO backs new U.S. approach...,'fully committed' nato backs new united states...,worldnews,2017-08-22,0
44915,LexisNexis withdrew two products from Chinese ...,lexisnexis withdrew two products from chinese ...,worldnews,2017-08-22,0
44916,Minsk cultural hub becomes haven from authorities,minsk cultural hub becomes haven from authorit...,worldnews,2017-08-22,0
44917,Vatican upbeat on possibility of Pope Francis ...,vatican upbeat on possibility of pope francis ...,worldnews,2017-08-22,0


Combined Corpus

In [21]:
cc_test = pd.read_csv(path + 'CC_test.csv', encoding="UTF-8")
cc_train=pd.read_csv(path + 'CC_train.csv', encoding="UTF-8")
cc=pd.concat([cc_train,cc_test], ignore_index=True)
cc= cc[cc['Statement'].notnull()]
cc= cc[cc['Label'].notnull()]  #dropping null rows

cc.rename({'Statement':'text','Label':'labels'}, axis=1, inplace=True)
cc=cc.convert_dtypes()
cc=cc[(cc['labels']=='1')|(cc['labels']=='0')]
cc['labels']=pd.to_numeric(cc['labels'])

#cleaning
cc['text']=clean_cc(cc['text'])


cc.to_csv(path + 'CC_may.csv')

In [20]:
cc

,text,labels
0,report: war looms - hundreds of american troop...,0
1,i walked through a preview of the artist pedro...,1
2,neo-con hypocrites leverage human rights again...,0
3,"a giant beef roast for the holidays is, for ma...",1
4,seeking to appeal to guests desire for new exp...,1
...,...,...
86547,medicinal substances that can effectively hel...,0
86548,tweet (newstarget) viruses that cause winter ...,0
86549,tweet (newstarget) the foundation of a health...,0
86550,when it comes to making a living and pursuing...,0


Kaggle Fake News

In [26]:
kaggle=pd.read_csv(path + 'kaggletrain.csv', encoding="UTF-8")
kaggle['text']=kaggle['title']+'. '+kaggle['author']+'. '+kaggle['text'] #concatenate title, author and text
kaggle.rename({'label': 'labels'}, axis=1, inplace=True)
kaggle=kaggle.dropna() #drop null rows


#cleaning
kaggle['text']=clean_text(kaggle['text'])

kaggle.to_csv(path + 'Kaggle_may.csv')

In [25]:
kaggle

,id,title,author,text,labels
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,house dem aide: we did not even see comey lett...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,"flynn: hillary clinton, big woman on campus - ...",0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,why the truth might get you fired. consortiumn...,1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,15 civilians killed in single us airstrike hav...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,iranian woman jailed for fictional unpublished...,1
...,...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,rapper t.i.: trump a ’poster child for white s...,0
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,"n.f.l. playoffs: schedule, matchups and odds -...",0
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,macy is said to receive takeover approach by h...,0
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"nato, russia to hold parallel exercises in bal...",1


Covid-19 Fake news

In [29]:
covid_train = pd.read_csv(path + 'Covid_Train.csv', encoding="UTF-8")
covid_val = pd.read_csv(path + 'Covid_Val.csv', encoding="UTF-8")
covid_test = pd.read_csv(path + 'Covid_Test_Label.csv', encoding="UTF-8")

covid_data=pd.concat([covid_train,covid_val,covid_test], ignore_index=True)
covid_data[['label']]=np.where(covid_data[['label']]=='real',0,1)


covid_data.rename({'tweet': 'text','label':'labels'}, axis=1, inplace=True)

#cleaning

covid_data['text']=clean_covid(covid_data['text'])


covid_data.to_csv(path + 'Covid_may.csv')

In [28]:
covid_data

,id,text,labels
0,1,the cdc currently reports 99031 deaths. in gen...,0
1,2,states reported 1121 deaths a small rise from ...,0
2,3,politically correct woman (almost) uses pandem...,1
3,4,#indiafightscorona : we have 1524 #covid testi...,0
4,5,populous states can generate large case counts...,0
...,...,...,...
10695,2136,#corona virus updates: state-wise details of t...,0
10696,2137,tonight 12(midnight) onwards disaster manageme...,1
10697,2138,296 new cases of #covid 19nigeria; plateau-85 ...,0
10698,2139,rt @cdcemergency: #dyk? @cdcgov one-stop shop ...,0


Politifact

In [11]:
#May 22 edit to use source, source quote and title
politifact = pd.read_csv(path + 'politifact1.csv', encoding="utf-8") #curators_article_title, curator_complete_article,curator_tags,   fact
politifact= politifact.replace(r'\n','', regex=True) 
politifact=politifact.convert_dtypes()
politifact[["sources_dates"]] =politifact[["sources_dates"]] .apply(lambda x:pd.to_datetime(x,errors='coerce',format="%Y-%m-%d",utc=True))

####concatenate source, article title and the claim
politifact['text']=politifact['sources']+'. '+politifact['curators_article_title']+' '+politifact['sources_quote']

false=politifact[politifact['fact']=='FALSE']
true=politifact[politifact['fact']=='TRUE']
politifact=pd.concat([false,true], ignore_index=True) #only select completely true and false articles
politifact.rename({'fact': 'labels'}, axis=1, inplace=True)
politifact=politifact[['text','labels']]
politifact[['labels']]=np.where(politifact[['labels']]=='TRUE',0,1) #true news is labelled as 0



politifact = politifact[politifact['text'].notnull()] #drop null rows
politifact['text']=politifact['text'].apply(str)


#this is the scraped part
scraped = pd.read_csv(path + 'political_fact_checker4.csv')  
scraped=scraped.convert_dtypes()
scraped[["Date"]] = scraped[["Date"]] .apply(lambda x:pd.to_datetime(x,errors='coerce',format="%Y-%m-%d"))
scraped['Link2']=[x.rsplit('/',2)[1] for x in scraped['Link']] #this is extracting the title from the Link

#####concatenate source, article title and the claim
scraped['text']=scraped['Source']+' '+scraped['Link2']+' '+scraped['Statement']

scraped=scraped.loc[scraped['Date']>='2021-04-18'] #only selecting recent data to not overlap with the previous dataset
false=scraped[scraped['Label']=='false']
true=scraped[scraped['Label']=='true'] #only select completely true and false articles
scraped=pd.concat([false,true], ignore_index=True)
scraped.rename({'Label': 'labels'}, axis=1, inplace=True)
scraped=scraped[['text','labels']]
scraped[['labels']]=np.where(scraped[['labels']]=='true',0,1)

politifact=pd.concat([politifact, scraped], ignore_index=True)
politifact = politifact[politifact['text'].notnull()]
politifact['text']=politifact['text'].apply(str)



#cleaning

politifact['text']=clean_text(politifact['text'])



politifact.to_csv(path + 'Politifact_may.csv')

In [12]:
politifact

,text,labels
0,"viral image. no, disneyland is not enforcing a...",1
1,"facebook posts. no, covid-19 relief law does n...",1
2,"tweets. no, black lives matter was not charged...",1
3,facebook posts. no evidence that burisma admit...,1
4,"viral image. no, this candace owens tweet is n...",1
...,...,...
7773,jeff jackson us-senate-candidate-right-about-n...,0
7774,kevin kiley true-san-francisco-had-far-more-dr...,0
7775,kirk cox under-new-virginia-law-pound-pot-will...,0
7776,van wanggaard yes-foxconn-largest-taxpayer-rac...,0
